This code is used to check the accuracy of the model

In [1]:
import warnings
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from bokeh.io import output_notebook
from bokeh.plotting import figure as bo_figure
from bokeh.plotting import show as bo_show
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import DataLoader
from torchvision.models import resnet34, resnet50
from resnet_cbam import resnet34_cbam, resnet50_cbam

warnings.filterwarnings('ignore')
output_notebook()
device = torch.device('cpu')
num_classes = 1

data_transform = transforms.Compose([
transforms.Resize(360),  # Scaling the image (Image), keeping the aspect ratio constant
transforms.Grayscale(1),
transforms.ToTensor(),  # Convert Image to Tensor, normalize to [0, 1]
transforms.Normalize(mean=0.0601, std=0.1734)  # Standardized to [-1, 1], specifying the mean and standard deviation
])

test_dataset = datasets.ImageFolder(root="E:/Thesis/code/final/target_data/test/",transform=data_transform) #Testdata file locations
# Data loader
test_loader = DataLoader(test_dataset,batch_size=1,shuffle=False)

"""
model = resnet50(pretrained=False, num_classes=num_classes)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.to(device)
"""
model = resnet34_cbam(pretrained=False, num_classes=num_classes)

def convert_target3(target):
    
    i = 0
    y = torch.zeros(target)
    y = y.to(torch.float32)
    for item in target:
        y = float(test_dataset.classes[int(item)])
    y = torch.tensor(y)
    y = torch.unsqueeze(y, dim=0)
    y = torch.unsqueeze(y, dim=1)
    # print(y)
    return y

def inverse_norm_to_porosity(norm_value):
    """ Converting normalized density values to porosity values """
    return 1 - (0.92+norm_value/10)/1.02

def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.fabs((y_true - y_pred)/((np.abs(y_true)+np.abs(y_pred))/2)).mean()

def nrmse(y_true, y_pred):
    """ Normalized Root Mean Squared Error """
    actual = np.array(y_true)
    return np.sqrt(mean_squared_error(y_true,y_pred)) / (actual.max() - actual.min())

def mae_percent(y_true, y_pred):
    return mean_absolute_error(y_true,y_pred)/ (sum(y_true)/len(y_true))

def check_accuracy(loader, model):
    """ Checking model accuracy """
    num_samples = 0
    model.load_state_dict(torch.load('res_34.pt'))
    model.eval()  # Adjust the model to eval mode
    with torch.no_grad():
        predictions = torch.zeros((loader.__len__()))
        y_real = torch.zeros((loader.__len__()))
        i = 0
        for x, y in loader:
            x = x
            y_real[i] = inverse_norm_to_porosity(convert_target3(y))
            predictions[i] = inverse_norm_to_porosity(model(x))
            # print('pre', predictions[i])
            # print('real', y_real[i])
            num_samples += 1
            i += 1


    fit_x = np.linspace(0,1,100).tolist()
    fit_y = np.linspace(0,1,100).tolist()
    predict_dot_x = y_real.numpy().tolist()
    predict_dot_y = predictions.numpy().tolist()
    MAE_loss = mean_absolute_error(predict_dot_x,predict_dot_y)
    print(f'Got {num_samples} with MAE_loss {MAE_loss:.6f}')
    MSE_loss = mean_squared_error(predict_dot_x,predict_dot_y)
    print(f'Got {num_samples} with MSE_loss {MSE_loss:.6f}')
    RMSE_loss = np.sqrt(mean_squared_error(predict_dot_x,predict_dot_y))
    print(f'Got {num_samples} with RMSE_loss {RMSE_loss:.6f}')
    NRMSE_loss = nrmse(predict_dot_x,predict_dot_y)
    print(f'Got {num_samples} with NRMSE_loss {NRMSE_loss:.6f}')
    SMAPE_loss = smape(predict_dot_x,predict_dot_y)
    print(f'Got {num_samples} with SMAPE_loss {SMAPE_loss:.6f}')
    R2_score = r2_score(predict_dot_x,predict_dot_y)
    print(f'Got {num_samples} with R2_score {R2_score:.6f}')
    MAE_percent = mae_percent(predict_dot_x,predict_dot_y)
    print(f'Got {num_samples} with MAE_percent {MAE_percent:.6f}')
    # plot use Bokeh
    p = bo_figure(plot_width=450, plot_height=400, y_range=[-0.001, 0.1], x_range=[-0.001, 0.1],
    x_axis_label='Porosity, experimental values', y_axis_label='Porosity, predicted values')
    p.circle(predict_dot_x, predict_dot_y, color="black")
    p.line(fit_x,fit_y, legend="Perfect fit", line_color="grey", alpha=0.5, line_dash="2 2")
    p.legend.location = "top_left"
    bo_show(p)
    return 



check_accuracy(test_loader, model)

Loading BokehJS ...

Got 103 with MAE_loss 0.002903
Got 103 with MSE_loss 0.000023
Got 103 with RMSE_loss 0.004766
Got 103 with NRMSE_loss 0.049106
Got 103 with SMAPE_loss 0.158551
Got 103 with R2_score 0.957453
Got 103 with MAE_percent 0.078165
